In [1]:
import pandas as pd
import numpy as np
import scipy as sc
import datetime
import os
import re

# DECLARING GLOBAL VARIABLES

constants = set([ 'COMM', 'DUPLEX', 'GARDEN', 'PH', 'RES', 'TH', 'UNIT'])
remove = set(['FLR', 'FL', 'UNIT', 'APT'])
mod_listings = [] # np.zeros((len(listings), len(listings.iloc[0])+1))
mod_closings = [] # np.zeros((len(closings), len(closings.iloc[0])+1))

In [2]:
# READING CSV INTO DATAFRAME

def start():
    listings = pd.read_csv('listings.csv', header='infer', sep=',',index_col=None)
    closings = pd.read_csv('closings.csv', header='infer', sep=',',index_col=None)
    listings['list_date'] = pd.to_datetime(listings['list_date'], format="%Y-%m-%d").dt.floor('d')
    closings['close_date'] = pd.to_datetime(closings['close_date'], format="%Y-%m-%d")
    closings['close_price'] = closings['close_price'].astype('int64')
    listings = listings.dropna()
    closings = closings.dropna()

#     print(listings.dtypes)
#     print()
#     print(closings.dtypes)
    print(listings.head())
    print(closings.head())
    return listings, closings

In [3]:
# VALIDATION AND PARSING FUNCTIONS

def date_validation(listing_date, closing_date):
    return (listing_date < closing_date)

def price_validation(list_price, close_price):
    if close_price < (list_price * 1.4):
        return True
    return False

def parse_digit(apt, unit_nums, apt_nums):
    
    # 105, 1106
    if len(apt) > 2:
        unit_nums.append(str(int(apt[:-2]))) # remaining digits
        apt_nums.append(str(int(apt[-2:]))) # last 2 digits
    
    # 5, 11
    else:
        unit_nums.append("N/A")
        apt_nums.append(apt)
    
    return unit_nums, apt_nums

def parse_alphanumeric(apt, unit_nums, apt_nums):
    
    # APT 10, APT10, 3FLR, 
    for string in remove:
        if string in apt:
            apt = apt.replace(string, "")
            return parse_apartment_number(apt, unit_nums, apt_nums)

    # PH03, GARDENA
    for string in constants:
        if string in apt:
            unit_nums.append(string)
            apt = apt.replace(string, "")
            apt_nums.append(apt)
            return unit_nums, apt_nums
    
    if len(apt) > 0:
        # N6, 31C regex extract char, append resp
        r1 = re.compile("([0-9]+)([a-zA-Z]+)")
        r2 = re.compile("([a-zA-Z]+)([0-9]+)")
        m1 = r1.match(apt)
        m2 = r2.match(apt)
#         l = 0

        if m1:
            unit_nums.append(m1.group(1))
            apt_nums.append(m1.group(2))
            l = len(m1.group(1)) + len(m1.group(2))
#             if(l > 0):
#                 unit_nums, apt_nums = parse_alphanumeric(m1[-l:], unit_nums, apt_nums)
        elif m2:
            unit_nums.append(m2.group(2))
            apt_nums.append(m2.group(1))
            l = len(m2.group(1)) + len(m2.group(2))
#             if(l > 0):
#                 unit_nums, apt_nums = parse_alphanumeric(m2[-l:], unit_nums, apt_nums)
    
    
    # Additive 2EF, 11D12E, 1213A
    
    return unit_nums, apt_nums

def parse_spec_chars(apt, unit_nums, apt_nums):
    
    if('-' in apt):
        apt = apt.replace('-','')
        return parse_apartment_number(apt, unit_nums, apt_nums) 
    
    elif(',' in apt):
        split_apt = apt.split(",")
        unit_nums0, apt_nums0 = parse_apartment_number(split_apt[0], unit_nums, apt_nums)
        unit_nums1, apt_nums1 = parse_apartment_number(split_apt[1], unit_nums, apt_nums)
        unit_nums = unit_nums0 + unit_nums1
        apt_nums = apt_nums0 + apt_nums1
        return unit_nums, apt_nums
    
    elif('/' in apt):
        split_apt = apt.split("/")
        
        # 1101/02/03 
        if len(split_apt[0]) > len(split_apt[1]):
            result = [split_apt[0]]
            for element in split_apt[1:]:
                result.append(split_apt[0][:-len(element)] + element)
            for item in result:
                apt_nums.append (item[len(split_apt[1]):])
                unit_nums.append(item[0:len(split_apt[1])])
        
        # 22/23A 2/3CD
        elif len(split_apt[1]) > len(split_apt[0]):
            result = []
            for element in split_apt[0:1]:
                result.append(element + split_apt[1][len(element):] )
                result.append(split_apt[1])
                for item in result:
                    apt_nums.append (item[len(split_apt[0]):])
                    unit_nums.append(item[0:len(split_apt[0])])
        
        # 2/3 22/C
        elif len(split_apt[1]) == len(split_apt[0]):
            unit_nums0, apt_nums0 = parse_apartment_number(split_apt[0], unit_nums, apt_nums)
            unit_nums1, apt_nums1 = parse_apartment_number(split_apt[1], unit_nums, apt_nums)
            unit_nums = unit_nums0 + unit_nums1
            apt_nums = apt_nums0 + apt_nums1
    
    return unit_nums, apt_nums

def parse_apartment_number(apt, unit_nums, apt_nums):
    
    if ('/' in apt or '-' in apt or ',' in apt):
        unit_nums, apt_nums = parse_spec_chars(apt, unit_nums, apt_nums)
    
    elif (apt.isdigit()):
        unit_nums, apt_nums = parse_digit(apt, unit_nums, apt_nums)

    elif (apt.isalnum()):
        apt = apt.upper()
        unit_nums, apt_nums = parse_alphanumeric(apt, unit_nums, apt_nums)

    if len(unit_nums) == len(apt_nums):
        return unit_nums, apt_nums
    
    return None, None

In [4]:
# MODIFYING DATAFRAMES - DENORMALIZATION

def generate_data_point(row_id, bldg_id, unit_nums, apt_nums, price, row_date, table_id, apa_number):
    
    # Table_id : 1 for Listings
    # Table_id : 2 for Closings
    for i in range(len(apt_nums)):
        list_id = row_id
        building_id = bldg_id
        unit = unit_nums[i]
        apt = apt_nums[i]
        price = price/len(apt_nums) # ASSUMPTION
        date = row_date
        apa = apa_number
        
        if(table_id==1):
            # append to mod_listings
            mod_listings.append(np.array([list_id,price, date, building_id, unit, apt, apa]))
            
        else:
            # append to mod_closings
            mod_closings.append(np.array([list_id,price, date, building_id, unit, apt, apa]))

def modify_listings(listings):
    for i in range(len(listings)):
        try:
            unit_nums, apt_nums = parse_apartment_number(listings.iloc[i]['apartment_number'], [], [])
            generate_data_point(listings.iloc[i]['listing_id'], 
                                listings.iloc[i]['building_id'], 
                                unit_nums,
                                apt_nums,
                                listings.iloc[i]['list_price'], 
                                listings.iloc[i]['list_date'], 1,
                                listings.iloc[i]['apartment_number'])
        except:
            print("excpet")
            print(listings.iloc[i]['listing_id'], listings.iloc[i]['apartment_number'])
    return mod_listings
    
def modify_closings(closings):
    for i in range(len(closings)):
        try:
            unit_nums, apt_nums = parse_apartment_number(closings.iloc[i]['apartment_number'], [], [])
            generate_data_point(closings.iloc[i]['closing_id'], 
                                closings.iloc[i]['building_id'], 
                                unit_nums,
                                apt_nums,
                                closings.iloc[i]['close_price'], 
                                closings.iloc[i]['close_date'], 2,
                                closings.iloc[i]['apartment_number'])
        except:
            print(closings.iloc[i]['closing_id'], closings.iloc[i]['apartment_number'])
    return mod_closings

In [5]:
# FUNCTION MAIN

listings, closings = start()
# modify_listings(listings)

list_cols = ["list_price", "list_date", "building_id", "unit", "apartment"]
close_cols = ["close_price", "close_date", "building_id", "unit", "apartment"]


# new_listings = pd.DataFrame(mod_listings)
# new_listings.columns = list_cols
# print(new_listings.head())

   listing_id  building_id apartment_number  list_price  list_date
0           1            7               PH     2300000 2018-08-02
1           2            7               14     2295000 2018-03-26
2           3           14             PH-4      685000 2017-11-21
3           4           14             1206      560000 2015-04-11
4           5           14             1407      585000 2015-04-14
   closing_id  building_id apartment_number  close_price close_date
0           1            6             COMM     19000000 2016-04-15
1           2            6         COMM,RES     30150000 2014-10-27
2           3            6              RES     26750000 2017-07-20
3           4            6              RES     26750000 2017-07-20
4           5            6              RES     26750000 2017-07-20


In [6]:
mod_listings = modify_listings(listings)
new_listings = pd.DataFrame(mod_listings)
print(new_listings)

           0          1          2       3    4   5     6
0          1  2300000.0 2018-08-02       7   PH        PH
1          2  2295000.0 2018-03-26       7  N/A  14    14
2          3   685000.0 2017-11-21      14   PH   4  PH-4
3          4   560000.0 2015-04-11      14   12   6  1206
4          5   585000.0 2015-04-14      14   14   7  1407
5          6   560000.0 2016-02-02      14   PH  03  PH03
6          7   575000.0 2017-02-02      14    5   8   508
7          8   925000.0 2018-04-30      14    4   9   409
8          9   629000.0 2014-09-18      14    9   4   904
9         10  1050000.0 2015-01-10      14   10   9  1009
10        11  1900000.0 2016-11-30      14    8   5   805
11        12  1995000.0 2015-12-03      14   13   5  1305
12        13   525000.0 2015-05-05      14    8   3   803
13        14   575000.0 2015-05-11      14   13   3  1303
14        15   659000.0 2016-11-28      14   13   2  1302
15        16   618000.0 2017-01-09      14   13   7  1307
16        17  

In [7]:
mod_closings = modify_closings(closings)
new_closings = pd.DataFrame(mod_closings)
print(new_closings)

           0             1          2       3     4   5         6
0          1  1.900000e+07 2016-04-15       6  COMM          COMM
1          2  7.537500e+06 2014-10-27       6  COMM      COMM,RES
2          2  1.884375e+06 2014-10-27       6   RES      COMM,RES
3          2  4.710938e+05 2014-10-27       6  COMM      COMM,RES
4          2  1.177734e+05 2014-10-27       6   RES      COMM,RES
5          4  2.675000e+07 2017-07-20       6   RES           RES
6          5  2.675000e+07 2017-07-20       6   RES           RES
7          6  2.200000e+06 2017-12-21       7   N/A   2       FL2
8          7  4.700000e+05 2016-04-13       7   N/A   5         5
9          8  5.801000e+05 2017-11-10      14     5   3       503
10         9  1.000000e+01 2018-07-13      14    12   8      1208
11        10  6.350000e+05 2018-02-28      14     8   4       804
12        11  1.285250e+06 2014-08-15      14     7   1       701
13        12  1.220000e+06 2018-08-10      14    14   5      1405
14        

In [8]:
# unit_nums = []
# apt_nums = []
# parse_apartment_number("COMM", unit_nums, apt_nums)
# parse_apartment_number("503", unit_nums, apt_nums)
# parse_apartment_number("31C", unit_nums, apt_nums)
# print(unit_nums)
# print(apt_nums)

In [9]:
print(new_listings[2][:20])

0    2018-08-02
1    2018-03-26
2    2017-11-21
3    2015-04-11
4    2015-04-14
5    2016-02-02
6    2017-02-02
7    2018-04-30
8    2014-09-18
9    2015-01-10
10   2016-11-30
11   2015-12-03
12   2015-05-05
13   2015-05-11
14   2016-11-28
15   2017-01-09
16   2017-01-19
17   2017-11-26
18   2016-10-11
19   2017-03-17
Name: 2, dtype: datetime64[ns]


In [12]:
inner_merged = pd.merge(new_listings, new_closings, on=[3,4,5],how='inner')
print(inner_merged[:30])


    0_x        1_x        2_x   3   4   5   6_x  0_y        1_y        2_y  \
0     4   560000.0 2015-04-11  14  12   6  1206   30   550000.0 2015-06-29   
1     5   585000.0 2015-04-14  14  14   7  1407   43   585000.0 2015-08-04   
2     6   560000.0 2016-02-02  14  PH  03  PH03   46   560000.0 2016-02-03   
3     7   575000.0 2017-02-02  14   5   8   508   28   565000.0 2017-06-05   
4     9   629000.0 2014-09-18  14   9   4   904   25   615000.0 2015-06-23   
5    10  1050000.0 2015-01-10  14  10   9  1009   29  1055000.0 2015-04-29   
6    11  1900000.0 2016-11-30  14   8   5   805   14  1855000.0 2017-02-16   
7    12  1995000.0 2015-12-03  14  13   5  1305   13  1900000.0 2016-08-18   
8    13   525000.0 2015-05-05  14   8   3   803   35   535000.0 2015-07-21   
9    14   575000.0 2015-05-11  14  13   3  1303   27   580000.0 2015-07-17   
10   15   659000.0 2016-11-28  14  13   2  1302   42   600000.0 2017-03-09   
11   16   618000.0 2017-01-09  14  13   7  1307   36   615000.0 

In [14]:
outer_merged = pd.merge(new_closings,new_listings,how='outer', on=[3,4,5])
outer_merged.loc[(outer_merged['1_x'].notnull()) & outer_merged['1_y'].notnull()]
print(outer_merged[:10])



   0_x           1_x        2_x   3     4  5       6_x   0_y       1_y  \
0  1.0  1.900000e+07 2016-04-15   6  COMM         COMM   NaN       NaN   
1  2.0  7.537500e+06 2014-10-27   6  COMM     COMM,RES   NaN       NaN   
2  2.0  4.710938e+05 2014-10-27   6  COMM     COMM,RES   NaN       NaN   
3  2.0  1.884375e+06 2014-10-27   6   RES     COMM,RES   NaN       NaN   
4  2.0  1.177734e+05 2014-10-27   6   RES     COMM,RES   NaN       NaN   
5  4.0  2.675000e+07 2017-07-20   6   RES          RES   NaN       NaN   
6  5.0  2.675000e+07 2017-07-20   6   RES          RES   NaN       NaN   
7  6.0  2.200000e+06 2017-12-21   7   N/A  2       FL2   NaN       NaN   
8  7.0  4.700000e+05 2016-04-13   7   N/A  5         5   NaN       NaN   
9  8.0  5.801000e+05 2017-11-10  14     5  3       503  22.0  575000.0   

         2_y  6_y  
0        NaT  NaN  
1        NaT  NaN  
2        NaT  NaN  
3        NaT  NaN  
4        NaT  NaN  
5        NaT  NaN  
6        NaT  NaN  
7        NaT  NaN  
8    